In [4]:
import sys, subprocess

def pip_install(pkg):
    print('Installing:', pkg)
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg])

pip_install('openai>=1.40.0')
pip_install('matplotlib>=3.7.0')
pip_install('PyYAML>=6.0.1')


Installing: openai>=1.40.0
Installing: matplotlib>=3.7.0
Installing: PyYAML>=6.0.1


In [5]:
import os
from getpass import getpass

api_key = os.getenv('OPENAI_API_KEY')
if not api_key:
    print('Enter your OpenAI API key (session only; not saved to disk):')
    api_key = getpass('OPENAI_API_KEY: ')
    os.environ['OPENAI_API_KEY'] = api_key

masked = 'SET (sk-***' + os.environ['OPENAI_API_KEY'][-6:] + ')'
print('OPENAI_API_KEY:', masked)


Enter your OpenAI API key (session only; not saved to disk):
OPENAI_API_KEY: SET (sk-***tmSokA)


In [37]:
import json
from typing import Any, Dict, List
from openai import OpenAI

# 사용할 OpenAI 모델 정의
MODEL = os.getenv("OPENAI_MODEL_REPORT", "gpt-4o-mini")

# LLM이 분류 기준으로 사용할 전략 태그 목록
ALLOWED_TAGS = [
    "panel", "login",
    "wordpress", "joomla", "wp-plugins", "cms",
    "tech",
    "exposure", "info-leak", "logs", "debug",
    "osint", "osint-social", "listing",
    "Miscellaneous" # 기타
]

def build_report_schema() -> Dict[str, Any]:
    """LLM이 생성할 보고서 데이터의 JSON 스키마를 정의합니다."""
    return {
        "name": "security_report_v3",
        "description": "Nuclei 스캔 결과를 분석하여 생성된 상세 보안 점검 보고서",
        "parameters": {
            "type": "object",
            "required": ["executive_summary", "severity_summary", "detailed_findings"],
            "properties": {
                "executive_summary": {
                    "type": "object",
                    "properties": {
                        "target_domain": {"type": "string"},
                        "total_findings": {"type": "integer"},
                        "highest_severity": {"type": "string"},
                        "overall_risk_level": {"type": "string", "enum": ["Critical", "High", "Medium", "Low", "Informational"]},
                        "summary_text": {"type": "string"}
                    }
                },
                "severity_summary": {
                    "type": "object",
                    "properties": {
                        "critical": {"type": "integer"}, "high": {"type": "integer"},
                        "medium": {"type": "integer"}, "low": {"type": "integer"}, "info": {"type": "integer"}
                    }
                },
                "detailed_findings": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "required": ["vulnerability_name", "severity", "exploitability", "strategic_tag", "description", "impact", "recommendation", "affected_url"],
                        "properties": {
                            "vulnerability_name": {"type": "string"},
                            "severity": {"type": "string", "enum": ["Critical", "High", "Medium", "Low", "Info"]},
                            "exploitability": {"type": "string", "enum": ["Easy", "Moderate", "Difficult"]},
                            "strategic_tag": {"type": "string", "enum": ALLOWED_TAGS},
                            "description": {"type": "string"},
                            "impact": {"type": "string"},
                            "recommendation": {"type": "string"},
                            "affected_url": {"type": "string"}
                        }
                    }
                }
            }
        }
    }

SYSTEM_PROMPT = f"""
당신은 최고의 사이버 보안 분석가입니다. Nuclei 스캔 결과를 분석하여 비전문가도 이해할 수 있는 명확한 보안 보고서를 생성하세요.

**보고서 생성 가이드라인:**
1.  **핵심 요약 (Executive Summary):** 반드시 `target_domain`, `total_findings` 등의 필드를 포함하는 **객체(object)**로 생성해야 합니다.
2.  **상세 분석 (Detailed Findings):** 각 취약점에 대해 아래 항목을 반드시 포함하여 분석하세요.
    - **strategic_tag (전략 태그):** 각 취약점이 아래 [허용 태그] 목록 중 어떤 것에 가장 가까운지 분류하세요. 예를 들어 'exposed-panels.yaml'은 'panel' 태그에 해당합니다. 적절한 태그가 없으면 'Miscellaneous'를 사용하세요.
    - **exploitability (공격 가능성):** 'Easy', 'Moderate', 'Difficult' 중 하나로 분류하세요.
    - **Description, Impact, Recommendation:** 비전문가가 이해하기 쉽게 작성하세요.
3.  **출력 형식:** 반드시 `security_report_v3` 함수의 인자 형식에 맞는 JSON 객체 하나만 반환해야 합니다.

[허용 태그]
{', '.join(ALLOWED_TAGS)}
"""

def generate_report_data_with_llm(nuclei_content: str) -> Dict[str, Any]:
    """Nuclei 스캔 결과를 받아 LLM을 통해 구조화된 보고서 데이터를 생성합니다."""
    client = OpenAI()
    report_tool_schema = build_report_schema()
    
    print("🚀 OpenAI API에 상세 보고서 생성을 요청합니다...")
    try:
        resp = client.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": f"아래 Nuclei 스캔 결과를 분석하여 보고서를 생성해 주세요:\\n\\n---\\n{nuclei_content}"}
            ],
            tools=[{"type": "function", "function": report_tool_schema}],
            tool_choice={"type": "function", "function": {"name": "security_report_v3"}},
            temperature=0.1
        )
        tool_call = resp.choices[0].message.tool_calls[0]
        report_data = json.loads(tool_call.function.arguments)
        print("✨ LLM으로부터 상세 분석 데이터를 성공적으로 받았습니다.")
        return report_data
    except Exception as e:
        print(f"🚨 LLM 호출 중 오류가 발생했습니다: {e}")
        return None

print("✅ LLM 호출 및 상세 데이터 생성 함수가 정의되었습니다.")

✅ LLM 호출 및 상세 데이터 생성 함수가 정의되었습니다.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# --- 1. 전체 태그 점검 현황 차트 (신규) ---
def create_all_tags_summary_chart(all_tags: list, findings_data: list, output_path: str = "all_tags_summary.png"):
    """전체 태그의 취약점 발견 건수를 시각화합니다."""
    
    # 모든 태그에 대해 발견 건수 집계
    tag_counts = {tag: 0 for tag in all_tags}
    for item in findings_data:
        tag = item.get('strategic_tag')
        if tag in tag_counts:
            tag_counts[tag] += 1
            
    df = pd.DataFrame(list(tag_counts.items()), columns=['tag', 'count']).sort_values(by='count')
    
    # 취약점 발견 유무에 따라 색상 결정
    colors = ['#ef9a9a' if count > 0 else '#c8e6c9' for count in df['count']]
    
    plt.figure(figsize=(12, 8))
    bars = plt.barh(df['tag'], df['count'], color=colors)
    
    # 막대 위에 숫자(발견 건수) 표시
    for bar in bars:
        width = bar.get_width()
        if width > 0:
            plt.text(width + 0.1, bar.get_y() + bar.get_height()/2, f'{width}', va='center')

    plt.title('Summary of All Scanned Tags', fontsize=16, fontweight='bold')
    plt.xlabel('Number of Vulnerabilities Found')
    plt.ylabel('Strategic Tag')
    plt.grid(axis='x', linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.savefig(output_path)
    print(f"📊 전체 태그 점검 현황 차트가 '{output_path}' 파일로 저장되었습니다.")
    plt.close()

# --- 2. 취약점 발견 태그 상세 분석 차트 (기존 개선) ---
def create_tag_severity_chart(findings_data: list, output_path: str = "tag_severity_chart.png"):
    """전략 태그별로 심각도 분포를 보여주는 누적 막대 차트를 생성합니다."""
    if not findings_data:
        print("📊 데이터가 없어 태그별 심각도 차트를 생성하지 않습니다.")
        return

    df = pd.DataFrame(findings_data)
    tag_severity_counts = df.groupby(['strategic_tag', 'severity']).size().unstack(fill_value=0)
    
    # 취약점이 발견된 태그만 필터링
    tag_severity_counts = tag_severity_counts[tag_severity_counts.sum(axis=1) > 0]
    
    if tag_severity_counts.empty:
        print("📊 취약점이 발견된 태그가 없어 상세 분석 차트를 생성하지 않습니다.")
        return

    severity_order = [s for s in ['Critical', 'High', 'Medium', 'Low', 'Info'] if s in tag_severity_counts.columns]
    tag_severity_counts = tag_severity_counts[severity_order]

    ax = tag_severity_counts.plot(
        kind='barh',
        stacked=True,
        figsize=(12, 8),
        color={'Critical': '#d62728', 'High': '#ff7f0e', 'Medium': '#ffbb78', 'Low': '#1f77b4', 'Info': '#aec7e8'}
    )

    plt.title('Detailed Analysis of Tags with Findings', fontsize=16, fontweight='bold')
    plt.xlabel('Number of Findings by Severity')
    plt.ylabel('Strategic Tag')
    plt.legend(title='Severity')
    plt.tight_layout()
    plt.savefig(output_path)
    print(f"📊 태그별 심각도 분포 차트가 '{output_path}' 파일로 저장되었습니다.")
    plt.close()

print("✅ 개선된 시각화 함수 2종이 정의되었습니다.")

✅ 개선된 시각화 함수 2종이 정의되었습니다!


In [46]:
def render_final_report(title: str, check_date: str, all_tags: list, report_data: dict, chart_paths: dict, output_path: str = "final_report2.md"):
    """개선된 템플릿과 시각화 자료로 최종 보고서를 생성합니다."""
    summary = report_data.get('executive_summary', {})
    findings = report_data.get('detailed_findings', [])
    target_domain = summary.get('target_domain', 'N/A') if isinstance(summary, dict) else "N/A"

    md = f"# 🛡️ {title}\n\n"
    md += f"| **점검 대상** | {target_domain} |\n"
    md += f"| :--- | :--- |\n"
    md += f"| **점검 일시** | {check_date} |\n\n"
    md += "---\n\n"
    
    md += "## 1. 개요 (Executive Summary)\n\n"
    if isinstance(summary, dict):
        md += f"> {summary.get('summary_text', '요약 정보 없음')}\n\n"
        md += f"- **총 발견 항목:** {summary.get('total_findings', len(findings))} 건\n"
        md += f"- **종합 위험 등급:** **{summary.get('overall_risk_level', 'N/A')}**\n\n"

    md += "## 2. 점검 결과 요약\n\n"
    
    # --- 1. 전체 태그 점검 현황 (표 추가) ---
    md += "### 2.1. 전체 태그 점검 현황\n\n"
    md += "> **설명:** 이번 점검에 사용된 모든 전략 태그와 태그별 취약점 발견 건수입니다. 이를 통해 전체 점검 범위와 취약점이 집중된 영역을 파악할 수 있습니다.\n\n"
    
    tag_counts = {tag: 0 for tag in all_tags}
    for item in findings:
        tag = item.get('strategic_tag')
        if tag in tag_counts:
            tag_counts[tag] += 1
            
    md += "| 전략 태그 (Strategic Tag) | 발견된 취약점 수 | 상태 |\n"
    md += "| :--- | :---: | :---: |\n"
    for tag, count in sorted(tag_counts.items()):
        status = "🔴 Finding" if count > 0 else "🟢 Clear"
        md += f"| `{tag}` | **{count}** | {status} |\n"
    md += "\n"
    md += f"![전체 태그 점검 현황]({chart_paths.get('all_tags_summary', '')})\n\n"
    
    # --- 2. 취약점 발견 태그 상세 분석 (차트) ---
    md += "### 2.2. 취약점 발견 태그 상세 분석\n\n"
    md += "> **설명:** 취약점이 발견된 태그들만 모아 각 태그에 어떤 심각도의 취약점들이 포함되어 있는지 보여줍니다. 특정 태그에 High, Critical 등급이 집중되어 있다면 해당 영역에 대한 시급한 조치가 필요함을 의미합니다.\n\n"
    md += f"![태그별 심각도 분포]({chart_paths.get('tag_severity', '')})\n\n"
    
    md += "---\n\n"
    md += "## 3. 상세 분석 결과\n\n"
    
    if not findings:
        md += "✅ 발견된 주요 보안 취약점이 없습니다.\n"
    else:
        # (상세 분석 결과 부분은 이전과 동일)
        severity_order = {"Critical": 5, "High": 4, "Medium": 3, "Low": 2, "Info": 1}
        sorted_findings = sorted(findings, key=lambda x: severity_order.get(x.get('severity', 'Info'), 0), reverse=True)
        for i, item in enumerate(sorted_findings):
            md += f"### 3.{i+1} {item.get('vulnerability_name', 'N/A')}\n\n"
            md += f"| 심각도 | 공격 가능성 | 전략 태그 |\n"
            md += f"| :---: | :---: | :--- |\n"
            md += f"| `{item.get('severity', 'N/A')}` | `{item.get('exploitability', 'N/A')}` | `{item.get('strategic_tag', 'N/A')}` |\n\n"
            md += f"- **발견 위치:** `{item.get('affected_url', 'N/A')}`\n\n"
            md += f"**상세 설명**\n{item.get('description', 'N/A')}\n\n"
            md += f"**예상 피해**\n{item.get('impact', 'N/A')}\n\n"
            md += f"**권고 조치**\n{item.get('recommendation', 'N/A')}\n\n---\n"
    
    with open(output_path, "w", encoding="utf-8") as f:
        f.write(md)
    
    print(f"📄 업그레이드된 최종 보고서가 '{output_path}' 파일로 성공적으로 생성되었습니다.")

print("✅ 업그레이드된 최종 보고서 생성 함수가 정의되었습니다.")

✅ 업그레이드된 최종 보고서 생성 함수가 정의되었습니다.


In [47]:
import datetime

# --- 1. 준비 단계 ---
input_file = "nuclei_new_output.md"
with open(input_file, "r", encoding="utf-8") as f:
    nuclei_content = f.read()
print(f"'{input_file}' 파일 내용을 읽었습니다.")

now = datetime.datetime.now()
check_date_str = now.strftime("%Y-%m-%d %H:%M:%S")
report_title = "test-target.com 웹 애플리케이션 정기 보안 점검"

# --- 2. LLM 분석 단계 ---
report_data = generate_report_data_with_llm(nuclei_content)

if report_data:
    # --- 3. 시각화 단계 ---
    chart_paths = {
        "all_tags_summary": "all_tags_summary.png",
        "tag_severity": "tag_severity_chart.png",
    }
    # ALLOWED_TAGS는 셀 4에서 정의되었습니다.
    create_all_tags_summary_chart(ALLOWED_TAGS, report_data['detailed_findings'], chart_paths['all_tags_summary'])
    create_tag_severity_chart(report_data['detailed_findings'], chart_paths['tag_severity'])

    # --- 4. 보고서 생성 단계 ---
    report_file = "final_report2.md"
    # ALLOWED_TAGS를 보고서 생성 함수에도 전달합니다.
    render_final_report(report_title, check_date_str, ALLOWED_TAGS, report_data, chart_paths, report_file)

    # --- 5. 결과 확인 ---
    print("\n--- 최종 보고서 미리보기 ---")
    with open(report_file, "r", encoding="utf-8") as f:
        print(f.read())
else:
    print("\n보고서 데이터 생성에 실패하여 프로세스를 중단합니다.")

'nuclei_new_output.md' 파일 내용을 읽었습니다.
🚀 OpenAI API에 상세 보고서 생성을 요청합니다...
✨ LLM으로부터 상세 분석 데이터를 성공적으로 받았습니다.
📊 전체 태그 점검 현황 차트가 'all_tags_summary.png' 파일로 저장되었습니다.
📊 태그별 심각도 분포 차트가 'tag_severity_chart.png' 파일로 저장되었습니다.
📄 업그레이드된 최종 보고서가 'final_report2.md' 파일로 성공적으로 생성되었습니다.

--- 최종 보고서 미리보기 ---
# 🛡️ test-target.com 웹 애플리케이션 정기 보안 점검

| **점검 대상** | wordpress-site.org |
| :--- | :--- |
| **점검 일시** | 2025-08-20 23:15:54 |

---

## 1. 개요 (Executive Summary)

> The scan identified several vulnerabilities in the WordPress site, with one high severity issue related to backup file disclosure.

- **총 발견 항목:** 3 건
- **종합 위험 등급:** **Medium**

## 2. 점검 결과 요약

### 2.1. 전체 태그 점검 현황

> **설명:** 이번 점검에 사용된 모든 전략 태그와 태그별 취약점 발견 건수입니다. 이를 통해 전체 점검 범위와 취약점이 집중된 영역을 파악할 수 있습니다.

| 전략 태그 (Strategic Tag) | 발견된 취약점 수 | 상태 |
| :--- | :---: | :---: |
| `Miscellaneous` | **0** | 🟢 Clear |
| `cms` | **0** | 🟢 Clear |
| `debug` | **0** | 🟢 Clear |
| `exposure` | **1** | 🔴 Finding |
| `info-leak` | **0** | 🟢 Clear |
| `j